## Running ISR from a notebook

Craig Lage - 16-Jun-22


    
        Here's an example of running ISR with a pipetask using the yaml file:

          isr:
            class: lsst.ip.isr.IsrTask
            config:
              connections.ccdExposure: raw
              connections.outputExposure: parameters.exposureName
              doWrite: true
              doOverscan: true
              doAssembleCcd: true
              doBias: true
              doVariance: true
              doLinearize: false
              doCrosstalk: false
              doBrighterFatter: false
              doDark: true
              doStrayLight: false
              doFlat: false
              doFringe: false
              doApplyGains: false
              doDefect: true
              doNanMasking: true
              doInterpolate: false
              doSaturation: false
              doSaturationInterpolation: false
              growSaturationFootprintSize: 0
     


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.daf.butler import Butler

In [ ]:
butler = Butler('/repo/main', collections=["LSSTCam/raw/all","LSSTCam/calib"])

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=False
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False


# Adjust these as needed and add as many more as you want

In [ ]:
isrTask = IsrTask(config=isrConfig)

In [ ]:
expId = 3021120600666
exp = butler.get('raw', detector=74, exposure=expId)
camera = butler.get('camera', instrument='LSSTCam')

In [ ]:
isrResult = isrTask.run(exp)

In [ ]:
fullArray = exp.image.array
print(fullArray.shape)

In [ ]:
assembledArray = isrResult.exposure.image.array
print(assembledArray.shape)

In [ ]:
print(4608/8, 4072/8, 4608/8 - 4072/8)
# 4 pre-scan pixels, 63 overscan pixels

In [ ]:
yVal = 1000
xStart1 = 503
xStop1 = 509
xStart1 = 509
xStop1 = 514
prescan = 4
overscan = 63
print(fullArray[1000, 567:577])
print(assembledArray[1000, 503:509])
print()
print(fullArray[1000, 635:645])
print(assembledArray[1000, 509:514])

In [ ]:
plt.figure(figsize=(8,16))
plt.suptitle(f"Assembly Test - Image {expId}", fontsize = 18)
plt.subplot(2,1,1)
plt.plot(fullArray[1000, 560:650], marker='o', label="Full Array")
plt.scatter(range(0,13), assembledArray[1000, 496:509], marker='x', s=200, color='red', label="Assembled Array-C10")
plt.scatter(range(80,90), assembledArray[1000, 509:519], marker='x', s=200, color='red', label="Assembled Array-C11")
plt.legend()
plt.subplot(2,1,2)
plt.plot(fullArray[1000, 560:650], marker='o', label="Full Array")
plt.scatter(range(0,13), assembledArray[1000, 496:509], marker='x', s=200, color='red', label="Assembled Array-C10")
plt.scatter(range(80,90), assembledArray[1000, 509:519], marker='x', s=200, color='red', label="Assembled Array-C11")
plt.ylim(95000, 97000)
plt.legend()
plt.savefig(f"/sdf/group/rubin/u/cslage/BOT_LSSTCam/image_assembly/Assembly_Test_{expId}_20Sep22.pdf")

In [ ]:
plt.plot(fullArray[0:100, 560], marker='x')
plt.plot(assembledArray[0:100, 560], marker='o')

In [ ]:
# Now look at the data with matplotlib
from matplotlib.colors import LogNorm

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = isrResult.exposure.image.array
img = plt.imshow(arr, norm=LogNorm(vmin=10, vmax=1000), interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)


In [ ]:
amps = camera[54].getAmplifiers()
for amp in amps:
    print(amp.getName(), amp.getBBox())

In [ ]:
# What about DETSEC? - E2V
filename = "/sdf/group/rubin/lsstdata/offline/instrument/LSSTCam-bot/storage/20211115/MC_C_20211115_000066/MC_C_20211115_000066_R13_S00.fits"
hdulist = pf.open(filename)

amps = camera[54].getAmplifiers()
print("hduNum ampName EXTNAME                 BBox                  DETSEC")
for i, amp in enumerate(amps):
    hduNum = i+1
    detsec = hdulist[hduNum].header['DETSEC']
    name = hdulist[hduNum].header['EXTNAME']
    print(hduNum,'     ', amp.getName(), name, amp.getBBox(), detsec)

In [ ]:
# What about DETSEC? - ITL
filename = "/sdf/group/rubin/lsstdata/offline/instrument/LSSTCam-bot/storage/20211115/MC_C_20211115_000066/MC_C_20211115_000066_R20_S02.fits"
hdulist = pf.open(filename)

amps = camera[74].getAmplifiers()
print("hduNum ampName EXTNAME                 BBox                  DETSEC")
for i, amp in enumerate(amps):
    hduNum = i+1
    detsec = hdulist[hduNum].header['DETSEC']
    name = hdulist[hduNum].header['EXTNAME']
    print(hduNum,'     ', amp.getName(), name, amp.getBBox(), detsec)

In [ ]:
hdulist[1].header